In [ ]:
"""
Purpose: To take in a whole neuron that could have any number of somas
and then to split it into multiple neuron objects

Pseudocode: 
1) Get all of the split suggestions
2) Split all of the limbs that need splitting
3) Once have split the limbs, split the neuron object into mutliple objects


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

INFO - 2021-01-03 23:20:55,029 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:20:55,030 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:20:55,031 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:20:55,034 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-03 23:20:55,035 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-03 23:20:55,044 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-03 23:20:55,291 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-03 23:20:55,375 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-03 23:20:55,615 - settings - Setting enable_python_native_blobs to True


# Picking the Neuron

In [8]:
minnie.Decomposition() & "n_error_limbs>3"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135269718693,0,0.25,=BLOB=,1182832,2399429,5,4,1,2,19,400,137,10420108.057891494,3856089.941250695,12989.77,128.84,128.84,598.532,584.44,3021,0.00028992021802615533,7.5525,6568007.796,189,0.0004599568231084968,15.984,132038841588.73079,26944413.464078583,12671.542449958893,20103.332043720184,698618209.464184,5676.1372
864691135307114822,0,0.25,=BLOB=,1206734,2471803,4,4,1,2,9,444,131,13571987.283598792,3796054.466195941,17976.693,102.864,102.864,431.865,361.569,4959,0.00036538495773516934,11.16891891891892,7456853.596,212,0.000665025796223236,23.392,202638130985.6534,15993487.39834086,14930.616036645832,27174.74982938547,955840240.4983652,6587.7301


In [9]:
segment_id = 864691135269718693 

In [11]:
neuron_obj = (minnie.Decomposition & dict(segment_id=segment_id)).fetch1("decomposition")
neuron_obj

Decompressing Neuron in minimal output mode...please wait


# Using a previously processed Neuron

In [ ]:
# file_loc = "/notebooks/test_neurons/Segmentation_3/Multi_Soma/864691135207768697_0_25"
# neuron_obj = nru.decompress_neuron(file_loc,file_loc)

# Visualizing the Neuron

In [12]:
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all",
                     mesh_whole_neuron=False)


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [ ]:
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict=dict(),
                     mesh_whole_neuron=True)

# Preview of How th Neuron Will Need to be Split

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)

In [ ]:
total_limbs_to_process = nru.error_limbs(neuron_obj)
total_limbs_to_process

# Getting the Split Suggestions

In [ ]:
limb_results = pru.multi_soma_split_suggestions(neuron_obj,
                                verbose=True,
                                )

In [ ]:
limb_results

In [ ]:
pru = reload(pru)
nviz = reload(nviz)
nviz.plot_split_suggestions_per_limb(neuron_obj,limb_results)

# Practice Cleaning the Skeleton

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict=dict(L0="all"),
                     return_color_dict=True)

In [ ]:
# getting the skeleton widths for all the branches
import matplotlib.pyplot as plt
sk_list = [sk.calculate_skeleton_distance(curr_branch.skeleton) for curr_branch in neuron_obj[0]]
plt.hist(sk_list)

In [ ]:
curr_limb = neuron_obj[0]

In [ ]:
curr_limb_sk_cleaned,removed_sk = sk.clean_skeleton(neuron_obj[0].skeleton,
                 distance_func=sk.skeletal_distance,
                 min_distance_to_junction=4001,
                 return_skeleton=True,
                 endpoints_must_keep=[k["starting_coordinate"] for k in curr_limb.all_concept_network_data],
                 print_flag=True,
                                        return_removed_skeletons=True)

In [ ]:
removed_sk

In [ ]:
nviz.plot_objects(curr_limb.mesh,
                 skeletons=[curr_limb_sk_cleaned] + removed_sk,
                 skeletons_colors=["black","red","red"],
                 #scatters=[k.reshape(-1,3) for k in removed_sk],
                 #scatter_size=0.5
                 )

# how to figure out which of the branches still remain?

In [ ]:
cleaned_branches = sk.decompose_skeleton_to_branches(curr_limb_sk_cleaned)
original_branches = [k.skeleton for k in curr_limb]
len(cleaned_branches),len(original_branches)

In [ ]:
from tqdm_utils import tqdm
from pykdtree.kdtree import KDTree

def map_between_branches_lists(branches_1,branches_2):
    """
    Purpose: 
    Will create a unique mapping of a branch
    in the first list to the best fitting branch in the second
    in terms of the most matching coordinates with a distance of 0
    
    Ex:
    cleaned_branches = sk.decompose_skeleton_to_branches(curr_limb_sk_cleaned)
    original_branches = [k.skeleton for k in curr_limb]
    map_between_branches_lists(original_branches,cleaned_branches)
    """
    old_to_new_branch_mapping = []

    for o_br in tqdm(original_branches):
        o_br_kd = KDTree(o_br.reshape(-1,3))

        n_matches = [len(np.where(o_br_kd.query(c_br.reshape(-1,3))[0]==0)[0]) for c_br in cleaned_branches]
        old_to_new_branch_mapping.append(np.argmax(n_matches))
        
    return np.array(old_to_new_branch_mapping)

sk.map_between_branches_lists(original_branches,cleaned_branches)
    

In [ ]:
poisson_mesh = tu.poisson_surface_reconstruction(curr_branch.mesh)
nviz.plot_objects(tu.split_significant_pieces(poisson_mesh)[0])

In [ ]:
curr_branch = neuron_obj[0][5]
curr_branch.width_new

In [ ]:
nviz.plot_limb_concept_network_2D(neuron_obj,
                                 node_colors=ret_col)

# Split the Neuron

In [ ]:
limb_results = pru.multi_soma_split_suggestions(neuron_obj,
                                plot_suggestions=True)

In [ ]:
limb_results

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict=dict(),
                     mesh_whole_neuron=True)

In [ ]:
limb_results

In [ ]:
nviz.plot_split_suggestions_per_limb(neuron_obj,
                                    limb_results=limb_results,
                                    scatter_size=0.7)

In [ ]:
neuron_list = pru.split_neuron(neuron_obj,
                plot_soma_limb_network=True,
                 plot_seperated_neurons=True)

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)